In [8]:
import numpy as np

In [9]:
class Node(object):
    """
    Base object for all inputs and outputs.
    """
    def __init__(self, value, grad):
        self.value = value
        self.gradient = grad

In [10]:
class MultiplyNode(object):
    """
    Multiplies two inputs
    """
    def forward(self, *arr):
        self.inputs = []
        
        for i in arr:
            self.inputs.append(i)
        
        self.output = Node(self.inputs[0].value * self.inputs[1].value, 0)
        return self.output
    
    def backward(self):
        self.inputs[0].gradient = self.inputs[1].value * self.output.gradient
        self.inputs[1].gradient = self.inputs[0].value * self.output.gradient

In [11]:
class AddNode(object):    
    """
    Adds two inputs x1 and x2.
    """
    def forward(self, nodes):
        self.inputs = nodes
             
        self.output = Node(self.inputs[0].value + self.inputs[1].value, 0)
        return self.output
    
    def backward(self):
        self.inputs[0].gradient = 1 * self.output.gradient
        self.inputs[1].gradient = 1 * self.output.gradient

In [12]:
class SigmoidNode(object):
    """
    Adds a sigmoid non-linearity to a single input
    """
    def forward(self, x):
        self.x = x
        self.output = Node(1/(1 + np.exp(-1 * self.x.value)), 0.0)
        return self.output
        
    def backward(self):
        s = 1/(1 + np.exp(-1 * self.x.value))
        self.x.gradient = (s * (1 - s)) * self.output.gradient

In [13]:
class Perceptron(object):
    def __init__(self, alpha =0.01,x1 = 0,x2=0, **args):
        ### Hyper parameters
        self.alpha = alpha
        ### Input and Output variables
        self.inputs = [Node(x1,0.0),Node(x2,0.0)]
        for i in args:
            input = Node(args[i],0.0)
            self.inputs.append(input)
        ### Initializing weights/bias to a random float between -1 and 1.
        self.weights = []
        for i in self.inputs:
            weight = Node(np.random.uniform(-1,1),0.0)
            self.weights.append(weight) 
        ### Initialize bias
        self.bias = Node(np.random.uniform(-1,1),0.0)
        ### Initialize operators nodes required 
        ### for processing the inputs within a perceptron
        self.initialize_operators()
        
    def initialize_operators(self):
        self.input_mul_w =[]
        for i in range(len(self.inputs)):
            self.input_mul_w.append(MultiplyNode())
        self.sum_wi = AddNode()
        self.sum_wi_b = AddNode()
        self.sigmoid = SigmoidNode()
        
        
    def forward(self):
        self.weighted_inputs = []
        for i in range(len(self.input_mul_w)):            
            input_mul_weight = self.input_mul_w[i].forward(self.inputs[i],self.weights[i])
            self.weighted_inputs.append(input_mul_weight)
        self.sum_w_i = self.sum_wi.forward(self.weighted_inputs)
        self.sum_w_i_b = self.sum_wi_b.forward([self.sum_w_i,self.bias])
        self.sigmoid.forward(self.sum_w_i_b)
        
    
    def backward(self):
        self.sigmoid.backward()
        self.sum_wi_b.backward()
        self.sum_wi.backward()
        for i in range(len(self.weighted_inputs)):
            self.input_mul_w[i].backward()
            
    def update(self):
        for i in range(len(self.weights)):
            self.weights[i].value -= self.alpha * self.weights[i].gradient
            
        self.bias.value -= self.alpha * self.bias.gradient

In [14]:
p1 = Perceptron(alpha = 0.1, x1 = 3, x2 = 5)
p2 = Perceptron()
# number of iterations
N = 100000
# expected output 
target = 0.3481972639817

for i in range(N):
    
    p1.forward()
    p2.x1=Node(p1.sigmoid.output.value, 0.0)
    p2.forward()
    p2.forward()
    p2.sigmoid.output.gradient = -2 * (target - p2.sigmoid.output.value)
    p2.backward()
    p2.update()
    p1.backward()
    p1.update()

p2.sigmoid.output.value

0.3481972639817122